<a href="https://colab.research.google.com/github/afiya-farjana/Machine-Learning-Lab/blob/main/ML_Lab_Final/ML_Lab_ProblemSet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Unzipping the dataset
from zipfile import ZipFile
with ZipFile ('/content/drive/MyDrive/CNN_dataset.zip', 'r') as zipOBJ:
  zipOBJ.extractall('CNN_Dataset')

In [4]:
#Dataset_Path
dataset_path = '/content/CNN_Dataset/CNN_dataset'
files = os.listdir(dataset_path)
print(files)

['HeadCT', 'ChestCT', 'Hand', 'AbdomenCT', 'CXR']


In [13]:
# Loading dataset using ImageDataGenerator and splitting it to train and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=45,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size= 45,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=45,
    class_mode='categorical',
    shuffle=False
)

Found 35000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.
Found 50000 images belonging to 5 classes.


In [14]:
#The CNN model
model = Sequential()

model.add(Conv2D(32, (3,3), activation = 'relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 6, 6, 128)       

In [15]:
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Training the model
hist = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=5,
                    validation_data=val_generator,
                    validation_steps=len(val_generator))

Epoch 1/5
778/778 [==============================] - 383s 491ms/step - loss: 0.0682 - accuracy: 0.9787 - val_loss: 0.0653 - val_accuracy: 0.9721
Epoch 2/5
778/778 [==============================] - 375s 482ms/step - loss: 0.0147 - accuracy: 0.9965 - val_loss: 0.0190 - val_accuracy: 0.9947
Epoch 3/5
778/778 [==============================] - 373s 479ms/step - loss: 0.0105 - accuracy: 0.9967 - val_loss: 0.0233 - val_accuracy: 0.9934
Epoch 4/5
778/778 [==============================] - 371s 477ms/step - loss: 0.0169 - accuracy: 0.9963 - val_loss: 0.0098 - val_accuracy: 0.9967
Epoch 5/5
778/778 [==============================] - 400s 514ms/step - loss: 0.0029 - accuracy: 0.9991 - val_loss: 0.0154 - val_accuracy: 0.9971


In [17]:
# Evaluating the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test loss: {loss:.2f}')
print(f'Test accuracy: {accuracy*100:.2f}%')

1112/1112 [==============================] - 123s 111ms/step - loss: 0.0062 - accuracy: 0.9984
Test loss: 0.01
Test accuracy: 99.84%


In [21]:
# Generating predictions
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

1112/1112 [==============================] - 123s 110ms/step


In [19]:
# Printing the confusion matrix & classification report
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))



Confusion Matrix
[[ 9989     0    11     0     0]
 [    0  9937     0    62     1]
 [    0     0 10000     0     0]
 [    0     0     0  9992     8]
 [    0     0     0     0 10000]]


In [20]:
print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

   AbdomenCT       1.00      1.00      1.00     10000
         CXR       1.00      0.99      1.00     10000
     ChestCT       1.00      1.00      1.00     10000
        Hand       0.99      1.00      1.00     10000
      HeadCT       1.00      1.00      1.00     10000

    accuracy                           1.00     50000
   macro avg       1.00      1.00      1.00     50000
weighted avg       1.00      1.00      1.00     50000

